# Efficient Yelp API Calls (Core)

### For this assignment, you will be working with the Yelp API.

### As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

### Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [13]:
!pip install yelpapi
!pip install tqdm

In [14]:
import json
with open('/Users/parri_nqdmzn3/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [15]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Yelp Search

In [16]:
location = 'San Antonio, Texas 78253'
term = 'chinese'

In [17]:
location.split(',')[0]

'San Antonio'

In [18]:
folder = 'Data/'

os.makedirs(folder, exist_ok=True)

JSON_FILE = folder+f"{location.split(',')[0]}-{term}.json"

In [19]:
JSON_FILE

'Data/San Antonio-chinese.json'

### Confirming Files

In [20]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/San Antonio-chinese.json not found. Saving empty list to file.


In [21]:
os.path.isfile(JSON_FILE)


True

### Load JSON

In [22]:
results = yelp_api.search_query(term = term, location = location)

In [23]:
type(results)


dict

In [24]:
len(results)

3

In [25]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [26]:
results['total']

88

In [27]:
results['region']

{'center': {'longitude': -98.79592895507812, 'latitude': 29.470648213030426}}

In [28]:
results['businesses']

[{'id': 'wCURMa_PGJ-pDCJYzdhDyg',
  'alias': 'rice-and-spicy-san-antonio',
  'name': 'Rice & Spicy',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/WjakyvEBVD860ADRz2hH8g/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/rice-and-spicy-san-antonio?adjust_creative=BiiAqVQ4IB4FQ6304SqKwg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=BiiAqVQ4IB4FQ6304SqKwg',
  'review_count': 70,
  'categories': [{'alias': 'thai', 'title': 'Thai'},
   {'alias': 'asianfusion', 'title': 'Asian Fusion'}],
  'rating': 4.5,
  'coordinates': {'latitude': 29.5271, 'longitude': -98.72229},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '11590 Galm Rd',
   'address2': 'Ste 108',
   'address3': None,
   'city': 'San Antonio',
   'zip_code': '78254',
   'country': 'US',
   'state': 'TX',
   'display_address': ['11590 Galm Rd', 'Ste 108', 'San Antonio, TX 78254']},
  'phone': '+12109995112',
  'display_phone': '(210) 999-5112',
  '

In [29]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wCURMa_PGJ-pDCJYzdhDyg,rice-and-spicy-san-antonio,Rice & Spicy,https://s3-media3.fl.yelpcdn.com/bphoto/Wjakyv...,False,https://www.yelp.com/biz/rice-and-spicy-san-an...,70,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 29.5271, 'longitude': -98.72229}","[pickup, delivery]",$$,"{'address1': '11590 Galm Rd', 'address2': 'Ste...",+12109995112,(210) 999-5112,9715.633559
1,ktyeFjGTfvsNimBxNswvfw,tjin-china-diner-san-antonio,Tjin China Diner,https://s3-media1.fl.yelpcdn.com/bphoto/JypG-0...,False,https://www.yelp.com/biz/tjin-china-diner-san-...,302,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 29.437046568496463, 'longitude': ...",[delivery],$$,"{'address1': '119 SW Lp 410', 'address2': 'Ste...",+12109982078,(210) 998-2078,14849.996793
2,yEbVfGVqp_yuLrCP02qkaQ,house-of-china-san-antonio,House of China,https://s3-media2.fl.yelpcdn.com/bphoto/A_3Iiy...,False,https://www.yelp.com/biz/house-of-china-san-an...,191,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",2.5,"{'latitude': 29.48848, 'longitude': -98.706555}","[pickup, delivery]",$,"{'address1': '5630 W Loop 1604 N', 'address2':...",+12105097888,(210) 509-7888,8875.582513
3,2SCe_RBPuAJauOSWedSJqA,lins-express-san-antonio,Lins Express,https://s3-media3.fl.yelpcdn.com/bphoto/UOJHRD...,False,https://www.yelp.com/biz/lins-express-san-anto...,55,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 29.484974, 'longitude': -98.72756}","[pickup, delivery]",$$,"{'address1': '11841 Alamo Ranch Pkwy', 'addres...",+12103621812,(210) 362-1812,6807.139513
4,zA74m8K0IYQMT4FfthcUGw,sichuan-house-san-antonio,Sichuan House,https://s3-media4.fl.yelpcdn.com/bphoto/7xU23U...,False,https://www.yelp.com/biz/sichuan-house-san-ant...,1014,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.5,"{'latitude': 29.471805, 'longitude': -98.623466}","[pickup, delivery]",$$,"{'address1': '3505 Wurzbach Rd', 'address2': '...",+12105099999,(210) 509-9999,16690.583465
5,oCCpQTfC0iZG-PZUFWTWlw,artea-culebra-commons-san-antonio,Artea - Culebra Commons,https://s3-media1.fl.yelpcdn.com/bphoto/fyI4Mi...,False,https://www.yelp.com/biz/artea-culebra-commons...,127,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 29.502815891590274, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '7010 W Loop 1604 N', 'address2':...",+12108883820,(210) 888-3820,9986.283750
6,BspvwcGxdFJz1aa72Z72gQ,royal-inn-oriental-cuisine-san-antonio,Royal Inn Oriental Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/7ye9ni...,False,https://www.yelp.com/biz/royal-inn-oriental-cu...,252,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 29.5160312652588, 'longitude': -9...","[pickup, delivery]",$,"{'address1': '5440 Babcock Rd', 'address2': 'S...",+12106910602,(210) 691-0602,20121.349158
7,hKSwFoVXwr9SpeiH6Na29A,tiger-s-chinese-cuisine-san-antonio,Tiger’s Chinese Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/onJixH...,False,https://www.yelp.com/biz/tiger-s-chinese-cuisi...,113,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.0,"{'latitude': 29.56257943164515, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '12730 W I-10 Frontage Rd', 'addr...",+12105385080,(210) 538-5080,22130.361454
8,h8RUrgghqoS1sn5ADC5gbg,sukhothai-thai-restaurant-express-3-san-antonio-2,Sukhothai Thai Restaurant Express 3,https://s3-media1.fl.yelpcdn.com/bphoto/ySjrc8...,False,https://www.yelp.com/biz/sukhothai-thai-restau...,3,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",5.0,"{'latitude': 29.485357, 'longitude': -98.724984}","[pickup, delivery]",NaN,"{'address1': '11720 Alamo Rnch Pkwy', 'address...",+12102815754,(210) 281-5754,7056.760605
9,ZtBR3SH_XRwGiYvQdZOMqw,asia-restaurant-san-antonio-2,Asia Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/60u_Dp...,False,https://www.yelp.com/biz/asia-restaurant-san-a...,115,"[{'